### Fetch Data from FBRef API

First get the API key

In [42]:
from python.fbref import FBRAPIDataset
import requests
import time

dataset = FBRAPIDataset()

Get the country codes and league ids

In [43]:
#target_countries = ['England', 'Spain', 'Italy', 'France', 'Germany']
#target_leagues = ['Premier League', 'La Liga', 'Serie A', 'Ligue 1', 'Bundesliga']
target_countries = ['England']
target_leagues = ['Premier League']

country_ids = dataset.get_country_ids(target_countries)
league_ids = dataset.get_league_ids(target_leagues, country_ids)
#season_ids = dataset.get_season_ids(league_ids) POSSIBLY USE THIS LATER, BUT FOR NOW WE ARE JUST USING 2018-2019 UNTIL 2023-2024
season_ids = ['2023-2024']

print("Country IDs:", country_ids, "league ids", league_ids, "season ids", season_ids)

Country IDs: ['ENG'] league ids [9] season ids ['2023-2024']


Fetch data for each league and season

In [44]:
# Loop over each combination of league and season
import json
import os

stats = []
for league_id in league_ids:
    for season_id in season_ids:
        stats.extend(dataset.fetch_team_stats(league_id, season_id))

# Save raw data to a JSON file for future use in the ./data directory
output_path = os.path.join('./data', "test_raw_team_stats.json")
with open(output_path, "w") as f:
    json.dump(stats, f, indent=2)

print(f"Raw data saved to {output_path}")

Raw data saved to ./data\test_raw_team_stats.json


Flatten dictionary and save to csv file

In [45]:
import pandas as pd

with open("./data/test_raw_team_stats.json", "r") as f:
    raw_data = json.load(f)

# Use the static method to flatten each record
flattened_data = [FBRAPIDataset.flatten_dict(record) for record in raw_data]

# Optionally, convert the flattened data to a DataFrame for easier inspection
df_flat = pd.DataFrame(flattened_data)
df_flat.to_csv("./data/test_team_stats.csv", index=False)
print("Structured data saved to test_team_stats.csv")

Structured data saved to test_team_stats.csv


### Attach ELO ratings to csv files as target column

In [46]:
import importlib
import python.elo

# Reload the module
importlib.reload(python.elo)

# Use the class directly from the module
elo_dataset = python.elo.ELODataset(['2022-2023', '2023-2024'])

# Load your team season stats CSV file
team_stats = pd.read_csv("data/test_team_stats.csv")

# Retrieve Elo ratings based on the team_stats data.
ratings_df = elo_dataset.get_ratings(team_stats)
print("ELO Ratings DataFrame:")
print(ratings_df.head())

# Merge the Elo ratings into the team_stats DataFrame.
merged_team_stats = elo_dataset.merge_ratings(team_stats)
print("Merged Dataset with Elo column:")
print(merged_team_stats.head())

# Save the merged dataset to a CSV file.
merged_team_stats.to_csv("data/test_team_stats_with_elo.csv", index=False)


ELO Ratings DataFrame:
  meta_data_team_name  season_id          Elo
0             Arsenal  2023-2024  1946.902832
1         Aston Villa  2023-2024  1770.435059
2         Bournemouth  2023-2024  1691.123657
3           Brentford  2023-2024  1711.083984
4            Brighton  2023-2024  1713.163208
Merged Dataset with Elo column:
  meta_data_team_id meta_data_team_name  stats_stats_roster_size  \
0          18bb7c10             Arsenal                       25   
1          8602292d         Aston Villa                       31   
2          4ba7cbea         Bournemouth                       30   
3          cd051869           Brentford                       28   
4          d07537b9            Brighton                       30   

   stats_stats_matches_played  stats_stats_ttl_gls  stats_stats_ttl_ast  \
0                          38                   86                   62   
1                          38                   72                   55   
2                          38      

### Normalize Data

In [47]:
import importlib
import python.dataset

# Force reload the module to get the latest changes
importlib.reload(python.dataset)

# Now import the function
from python.dataset import load_and_normalize_data

# Load and normalize the data
normalized_data = load_and_normalize_data(
    csv_path="data/test_team_stats_with_elo.csv", 
    output_path="data/test_team_stats_with_elo_normalized.csv"
)

# Display the first few rows to verify
print(normalized_data.head())


  meta_data_team_id meta_data_team_name  league_id  season_id  \
0          18bb7c10             Arsenal          9  2023-2024   
1          8602292d         Aston Villa          9  2023-2024   
2          4ba7cbea         Bournemouth          9  2023-2024   
3          cd051869           Brentford          9  2023-2024   
4          d07537b9            Brighton          9  2023-2024   

   stats_stats_roster_size  stats_stats_matches_played  stats_stats_ttl_gls  \
0                -1.388420                         0.0             1.568577   
1                 0.694210                         0.0             0.728804   
2                 0.347105                         0.0            -0.470873   
3                -0.347105                         0.0            -0.350905   
4                 0.347105                         0.0            -0.590841   

   stats_stats_ttl_ast  stats_stats_ttl_non_pen_gls  stats_stats_ttl_xg  ...  \
0             1.404585                     1.403081   

Split Data

In [48]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


# Quick info about the data
print("Full dataset shape:", data.shape)
print("Columns in data:")
print(normalized_data.columns.tolist())

# Identify all numeric columns and then select those for features (excluding the Elo target)
numeric_cols = normalized_data.select_dtypes(exclude=['object']).columns.tolist()
features = [col for col in numeric_cols if col != "Elo"]
print("\nNumeric feature columns (excluding Elo):")
print(features)

# Split data into 70% train and 30% temporary set (for validation and test)
train_data, temp_data = train_test_split(normalized_data, test_size=0.30, random_state=42, shuffle=True)

# Further split the temporary set equally into 15% validation and 15% test sets
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42, shuffle=True)

print("\nDataset split sizes:")
print("Training set shape:", train_data.shape)
print("Validation set shape:", val_data.shape)
print("Test set shape:", test_data.shape)

# Split each dataset into X (features) and y (target: Elo)
x_train = train_data[features]
y_train = train_data["Elo"]

x_val = val_data[features]
y_val = val_data["Elo"]

x_test = test_data[features]
y_test = test_data["Elo"]

# Print the shapes of the resulting splits to verify
print("\nFeature and target shapes:")
print("X_train:", x_train.shape, "y_train:", y_train.shape)
print("X_val:", x_val.shape, "y_val:", y_val.shape)
print("X_test:", x_test.shape, "y_test:", y_test.shape)

# Display the first few rows from each split for inspection
print("\nFirst few rows of X_train:")
print(x_train.head())
print("\nFirst few rows of y_train:")
print(y_train.head())

print("\nFirst few rows of X_val:")
print(x_val.head())
print("\nFirst few rows of y_val:")
print(y_val.head())

print("\nFirst few rows of X_test:")
print(x_test.head())
print("\nFirst few rows of y_test:")
print(y_test.head())


Full dataset shape: (20, 169)
Columns in data:
['meta_data_team_id', 'meta_data_team_name', 'league_id', 'season_id', 'stats_stats_roster_size', 'stats_stats_matches_played', 'stats_stats_ttl_gls', 'stats_stats_ttl_ast', 'stats_stats_ttl_non_pen_gls', 'stats_stats_ttl_xg', 'stats_stats_ttl_non_pen_xg', 'stats_stats_ttl_xag', 'stats_stats_ttl_pk_made', 'stats_stats_ttl_pk_att', 'stats_stats_ttl_yellow_cards', 'stats_stats_ttl_red_cards', 'stats_stats_ttl_carries_prog', 'stats_stats_ttl_passes_prog', 'stats_stats_avg_gls', 'stats_stats_avg_ast', 'stats_stats_avg_non_pen_gls', 'stats_stats_avg_xg', 'stats_stats_avg_xag', 'stats_stats_avg_non_pen_xg', 'stats_keepers_ttl_gls_ag', 'stats_keepers_avg_gls_ag', 'stats_keepers_sot_ag', 'stats_keepers_ttl_saves', 'stats_keepers_save_pct', 'stats_keepers_clean_sheets', 'stats_keepers_clean_sheet_pct', 'stats_keepers_pk_att_ag', 'stats_keepers_pk_made_ag', 'stats_keepers_pk_saved', 'stats_keepers_pk_miss_ag', 'stats_keepers_pk_save_pct', 'stats_kee

In [49]:
import torch
from torch.utils.data import TensorDataset
import pandas as pd

# Example: Assuming x_train, y_train, x_val, y_val, x_test, y_test are already loaded

# Convert training data
X_train_tensor = torch.tensor(x_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)

# Convert validation data
X_val_tensor = torch.tensor(x_val.values, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).view(-1, 1)

# Convert test data
X_test_tensor = torch.tensor(x_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# Create TensorDatasets for each split
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)


### Model Training and Evaluation

In [50]:
import torch

importlib.reload(python.model)
from python.model import SimpleNN, train_model


# Instantiate the model (input dimension based on x_train)
model = SimpleNN(X_train_tensor.shape[1])

# Define hyperparameters in a dictionary (adjust as needed)
hyperparams = {
    "batch_size": 32,
    "epochs": 1000,
    "learning_rate": 0.001,
    "print_every": 10,
    "best_model_path": "best_model.pth"
}

# Train the model using the training and validation datasets
model, validation_losses = train_model(model, train_dataset, val_dataset, hyperparams)


Training epochs:   0%|          | 0/1000 [00:00<?, ?it/s]

Epoch [1]: New best model saved with Validation Loss: 2920707.2500

Training epochs:   0%|          | 4/1000 [00:00<00:51, 19.32it/s]


Epoch [2]: New best model saved with Validation Loss: 2920546.2500
Epoch [3]: New best model saved with Validation Loss: 2920391.7500
Epoch [4]: New best model saved with Validation Loss: 2920246.2500


Training epochs:   3%|▎         | 30/1000 [00:00<00:10, 92.05it/s]

...

Epoch [945]: New best model saved with Validation Loss: 548819.4375
Epoch [946]: New best model saved with Validation Loss: 548813.5000
Epoch [947]: New best model saved with Validation Loss: 548808.0000
Epoch [948]: New best model saved with Validation Loss: 548802.5000
Epoch [949]: New best model saved with Validation Loss: 548796.9375
Epoch [950/1000], Training Loss: 0.4187, Validation Loss: 548791.1875
Epoch [950]: New best model saved with Validation Loss: 548791.1875
Epoch [951]: New best model saved with Validation Loss: 548785.9375
Epoch [952]: New best model saved with Validation Loss: 548780.6250
Epoch [953]: New best model saved with Validation Loss: 548775.1875
Epoch [954]: New best model saved with Validation Loss: 548770.0000
Epoch [955]: New best model saved with Validation Loss: 548764.9375
Epoch [956]: New best model saved with Validation Loss: 548759.7500
Epoch [957]: New best model saved with Validation Loss: 548754.7500
Epoch [958]: New best model saved with Validat

Training epochs:  99%|█████████▉| 994/1000 [00:07<00:00, 125.22it/s]

Epoch [972]: New best model saved with Validation Loss: 548685.8125
Epoch [973]: New best model saved with Validation Loss: 548681.5625
Epoch [974]: New best model saved with Validation Loss: 548677.5000
Epoch [975]: New best model saved with Validation Loss: 548673.1875
Epoch [976]: New best model saved with Validation Loss: 548669.4375
Epoch [977]: New best model saved with Validation Loss: 548665.5000
Epoch [978]: New best model saved with Validation Loss: 548661.4375
Epoch [979]: New best model saved with Validation Loss: 548657.6875
Epoch [980/1000], Training Loss: 0.1966, Validation Loss: 548653.8750
Epoch [980]: New best model saved with Validation Loss: 548653.8750
Epoch [981]: New best model saved with Validation Loss: 548650.0000
Epoch [982]: New best model saved with Validation Loss: 548646.1875
Epoch [983]: New best model saved with Validation Loss: 548642.4375
Epoch [984]: New best model saved with Validation Loss: 548638.6875
Epoch [985]: New best model saved with Validat

Training epochs: 100%|██████████| 1000/1000 [00:07<00:00, 135.82it/s]

Epoch [997]: New best model saved with Validation Loss: 548595.1250
Epoch [998]: New best model saved with Validation Loss: 548592.1875
Epoch [999]: New best model saved with Validation Loss: 548589.0000
Epoch [1000/1000], Training Loss: 0.1173, Validation Loss: 548586.2500
Epoch [1000]: New best model saved with Validation Loss: 548586.2500


In [51]:
from python.model import evaluate_model
import torch

# Load the best model state (if not already loaded)
model.load_state_dict(torch.load(hyperparams["best_model_path"]))
model.eval()

# Evaluate the model on the test dataset
test_loss = evaluate_model(model, test_dataset, hyperparams)
print(f"Test Loss: {test_loss:.4f}")


Test Loss: 597989.9375
